In [2]:
import xarray as xr
import xarray.plot as xplt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature


# Make a climatology of HRRR hdw

In [51]:
#load in one hrrr file to get the dimensions
ds= xr.open_dataset('./pygraf/Processed_HRRR_2020072500.nc')
ds = ds.drop_vars(['temp_2m', 'q_2m','gust_sfc','veggie','dewpt','weasd','soilm','esat_2m','e_2m','vpd_2m','dewpt_depression_2m',
                   'hwp','veg_term','gust_max_term','dd_term','mois_term','snowc_term','uwind_10m','vwind_10m','wind_speed'])

#dates of all the days, since we are doing daily averaging
dates = pd.date_range('2020-07-25', '2020-10-31', freq='D')
ds = ds.assign_coords({'time': dates}) #assign coords so we can resample along time

hdw = np.zeros((len(dates), len(ds['grid_yt'].values), len(ds['grid_xt'].values)))
hwp = np.zeros((len(dates), len(ds['grid_yt'].values), len(ds['grid_xt'].values)))

for ii in range(len(dates)):
    day = dates[ii]
    print(day)
    time_hour = pd.date_range(start=day, periods=24,freq='h')
    fnames_hour = './pygraf/Processed_HRRR_'+time_hour.strftime('%Y%m%d%H')+'.nc'
    ds_hour = xr.open_mfdataset(fnames_hour.values,concat_dim='time',combine='nested',compat='override', coords='all')
    ds_hour = ds_hour.assign_coords({'time': time_hour}) #assign coords so we can resample along time
    hdw_hourly = ds_hour['wind_speed']*ds_hour['vpd_2m']
    hdw[ii,:,:] = np.nanmean(hdw_hourly,axis=0)
    hwp[ii,:,:] = np.nanmean(ds_hour['hwp'].values,axis=0)

    
ds = ds.assign({'hdw':(('time', 'grid_yt', 'grid_xt'), hdw), 
                'hwp':(('time', 'grid_yt', 'grid_xt'), hwp)})
print(ds)

2020-07-25 00:00:00
2020-07-26 00:00:00
2020-07-27 00:00:00
2020-07-28 00:00:00
2020-07-29 00:00:00
2020-07-30 00:00:00
2020-07-31 00:00:00
2020-08-01 00:00:00
2020-08-02 00:00:00
2020-08-03 00:00:00
2020-08-04 00:00:00
2020-08-05 00:00:00
2020-08-06 00:00:00
2020-08-07 00:00:00
2020-08-08 00:00:00
2020-08-09 00:00:00
2020-08-10 00:00:00
2020-08-11 00:00:00
2020-08-12 00:00:00
2020-08-13 00:00:00
2020-08-14 00:00:00
2020-08-15 00:00:00
2020-08-16 00:00:00
2020-08-17 00:00:00
2020-08-18 00:00:00
2020-08-19 00:00:00
2020-08-20 00:00:00
2020-08-21 00:00:00
2020-08-22 00:00:00
2020-08-23 00:00:00
2020-08-24 00:00:00
2020-08-25 00:00:00
2020-08-26 00:00:00
2020-08-27 00:00:00
2020-08-28 00:00:00
2020-08-29 00:00:00
2020-08-30 00:00:00
2020-08-31 00:00:00
2020-09-01 00:00:00
2020-09-02 00:00:00
2020-09-03 00:00:00
2020-09-04 00:00:00
2020-09-05 00:00:00
2020-09-06 00:00:00
2020-09-07 00:00:00
2020-09-08 00:00:00
2020-09-09 00:00:00
2020-09-10 00:00:00
2020-09-11 00:00:00
2020-09-12 00:00:00


In [53]:
print(ds['hdw'].quantile([0,.28,.57,.84,.98,1]))
#print(fwi.quantile([0,.28,.57,.84,.98,1]).values)

<xarray.DataArray 'hdw' (quantile: 6)>
array([  1.25411895,  57.37825222, 101.42733978, 170.64740157,
       292.08609078, 968.76917286])
Coordinates:
  * quantile  (quantile) float64 0.0 0.28 0.57 0.84 0.98 1.0


In [14]:
dates = pd.date_range('2020-07-25', '2020-07-26', freq='h')

fnames = './pygraf/Processed_HRRR_'+dates.strftime('%Y%m%d%H')+'.nc'

ds = xr.open_mfdataset(fnames.values,concat_dim='time',combine='nested',compat='override', coords='all')
ds = ds.assign_coords({'time': dates}) #assign coords so we can resample along time
print(ds)

<xarray.Dataset>
Dimensions:              (grid_x: 1798, grid_xt: 1797, grid_y: 1058, grid_yt: 1057, time: 25)
Coordinates:
  * grid_x               (grid_x) int64 0 1 2 3 4 5 ... 1793 1794 1795 1796 1797
  * grid_y               (grid_y) int64 0 1 2 3 4 5 ... 1053 1054 1055 1056 1057
  * grid_xt              (grid_xt) int64 0 1 2 3 4 ... 1792 1793 1794 1795 1796
  * grid_yt              (grid_yt) int64 0 1 2 3 4 ... 1052 1053 1054 1055 1056
  * time                 (time) datetime64[ns] 2020-07-25 ... 2020-07-26
Data variables: (12/24)
    grid_lon             (time, grid_y, grid_x) float32 dask.array<chunksize=(1, 1058, 1798), meta=np.ndarray>
    grid_lat             (time, grid_y, grid_x) float32 dask.array<chunksize=(1, 1058, 1798), meta=np.ndarray>
    grid_lont            (time, grid_yt, grid_xt) float32 dask.array<chunksize=(1, 1057, 1797), meta=np.ndarray>
    grid_latt            (time, grid_yt, grid_xt) float32 dask.array<chunksize=(1, 1057, 1797), meta=np.ndarray>
    temp_

In [17]:
ds

<xarray.Dataset>
Dimensions:              (grid_x: 1798, grid_xt: 1797, grid_y: 1058, grid_yt: 1057, time: 25)
Coordinates:
  * grid_x               (grid_x) int64 0 1 2 3 4 5 ... 1793 1794 1795 1796 1797
  * grid_y               (grid_y) int64 0 1 2 3 4 5 ... 1053 1054 1055 1056 1057
  * grid_xt              (grid_xt) int64 0 1 2 3 4 ... 1792 1793 1794 1795 1796
  * grid_yt              (grid_yt) int64 0 1 2 3 4 ... 1052 1053 1054 1055 1056
  * time                 (time) datetime64[ns] 2020-07-25 ... 2020-07-26
Data variables: (12/24)
    grid_lon             (time, grid_y, grid_x) float32 dask.array<chunksize=(1, 1058, 1798), meta=np.ndarray>
    grid_lat             (time, grid_y, grid_x) float32 dask.array<chunksize=(1, 1058, 1798), meta=np.ndarray>
    grid_lont            (time, grid_yt, grid_xt) float32 dask.array<chunksize=(1, 1057, 1797), meta=np.ndarray>
    grid_latt            (time, grid_yt, grid_xt) float32 dask.array<chunksize=(1, 1057, 1797), meta=np.ndarray>
    temp_2m              (time, grid_yt, grid_xt) float32 dask.array<chunksize=(1, 1057, 1797), meta=np.ndarray>
    q_2m                 (time, grid_yt, grid_xt) float32 dask.array<chunksize=(1, 1057, 1797), meta=np.ndarray>
    ...                   ...
    dd_term              (time, grid_yt, grid_xt) float32 dask.array<chunksize=(1, 1057, 1797), meta=np.ndarray>
    mois_term            (time, grid_yt, grid_xt) float32 dask.array<chunksize=(1, 1057, 1797), meta=np.ndarray>
    snowc_term           (time, grid_yt, grid_xt) float32 dask.array<chunksize=(1, 1057, 1797), meta=np.ndarray>
    uwind_10m            (time, grid_yt, grid_xt) float32 dask.array<chunksize=(1, 1057, 1797), meta=np.ndarray>
    vwind_10m            (time, grid_yt, grid_xt) float32 dask.array<chunksize=(1, 1057, 1797), meta=np.ndarray>
    wind_speed           (time, grid_yt, grid_xt) float32 dask.array<chunksize=(1, 1057, 1797), meta=np.ndarray>